# Data Preprocessing and EDA
- combine headlines with financial data from Yahoo Finance
- separated headlines (data/combined_headlines.csv) by source (Guardian headlines may be too UK-centric to be relevant - tbd)
- examine correlation sentiment per existing model (mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis) vs. S&P500 trend?

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

### Load the data into Pandas dataframes

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive/')

In [ ]:
#data_cnbc = pd.read_csv("/content/drive/MyDrive/SP500-Predictor/cnbc_headlines.csv")
#data_guardian = pd.read_csv("/content/drive/MyDrive/SP500-Predictor/guardian_headlines.csv")
#data_reuters = pd.read_csv("/content/drive/MyDrive/SP500-Predictor/reuters_headlines.csv")

data_cnbc = pd.read_csv("../data/cnbc_headlines.csv")
data_guardian = pd.read_csv("../data/guardian_headlines.csv")
data_reuters = pd.read_csv("../data/reuters_headlines.csv")

In [ ]:
data_cnbc["Source"] = "CNBC"
data_guardian["Source"] = "Guardian"
data_reuters["Source"] = "Reuters"

In [ ]:
data_cnbc.head()

In [ ]:
data_guardian.head()

In [ ]:
data_reuters.head()

In [ ]:
data = pd.concat([data_cnbc, data_guardian, data_reuters], axis=0, ignore_index=True)

In [ ]:
data

In [ ]:
data.info()

### Convert date column to datetime

In [ ]:
data['Date'] = pd.to_datetime(data['Date'])
data['Date'] = data['Date'].dt.tz_localize(None)

### Determine date range covered by the dataset

In [ ]:
print("Start: ", data["Date"].min().date())
print("End:   ", data["Date"].max().date())

In [ ]:
plt.plot(data["Date"])

### Sort data chronologically (will make it easier to verify that financial data is added correctly)

In [ ]:
data.sort_values(by='Date', inplace=True)
data = data.reset_index(drop=True)

In [ ]:
data

In [ ]:
plt.plot(data["Date"])

## Collect financial data from Yahoo Finance

In [ ]:
import yfinance as yf

In [ ]:
ticker = yf.Ticker("SPY")
start_date = data["Date"].min().date() - pd.Timedelta(days=2) # buffer to account for weekends
end_date = data["Date"].max().date() + pd.Timedelta(days=10)
spy_data = ticker.history(start=start_date, end=end_date)
spy_data = spy_data.reset_index()
spy_data["Date"] = pd.to_datetime(spy_data["Date"])
spy_data["Date"] = spy_data["Date"].dt.tz_localize(None)

In [ ]:
spy_data

### Add closing prices for next three trading days

In [ ]:
spy_data = spy_data[["Date", "Close"]]
spy_data["CloseD+1"] = spy_data["Close"].shift(-1)
spy_data["CloseD+2"] = spy_data["Close"].shift(-2)
spy_data["CloseD+3"] = spy_data["Close"].shift(-3)
spy_data

In [ ]:
spy_data = spy_data.set_index("Date").asfreq("D", method="ffill").reset_index()
spy_data

### Merge stock market data into original DataFrame

In [ ]:
merged_data = pd.merge(data, spy_data[["Date", "Close", "CloseD+1", "CloseD+2", "CloseD+3"]], on="Date", how="left")
merged_data.sample(10)

### Calculate percentage shift in value 1, 2 and 3 trading days after the current date

In [ ]:
for i in range(1, 4):
    spy_data[f"PercentageD+{i}"] = ( ( spy_data[f"CloseD+{i}"] - spy_data["Close"] ) / spy_data["Close"] ) * 100

In [ ]:
spy_data

In [ ]:
import seaborn as sns

plt.figure(figsize=(10, 6))

# Using histplot to show histogram and KDE
sns.displot(spy_data['PercentageD+1'])

# Or use kdeplot for only KDE
# sns.kdeplot(df['percent_values'], shade=True)

# Plot customization
plt.title('Distribution of Percent Values')
plt.xlabel('Percent Values')
plt.ylabel('Density')
plt.show()

In [ ]:
spy_data[["PercentageD+1","PercentageD+2","PercentageD+3"]].describe()

In [ ]:
spy_data[spy_data["PercentageD+1"] < -0.5]

### Add category labels characterizing observed trends

Focus on TrendD+1; intuitively, this is most likely to be correlated with the sentiment of individual headlines.

Changes between -0.5% and +0.5% in a day are very common. Therefore, define trends as follows:

 - 'decrease': percentage shift < -0.5% --> 0
 - 'maintain': percentage shift between -0.5% and +0.5% --> 1
 - 'increase': percentage shift > 0.5% --> 2

In [ ]:
for i in range(1, 4):
    spy_data[f"TrendD+{i}"] = spy_data[f"PercentageD+{i}"].apply(
        lambda x: 1 if -0.5 <= x <= 0.5
        else (0 if x < -0.5
        else 2)
    )

In [ ]:
spy_data

### Merge financial data into original dataframe

In [ ]:
data = data.merge(spy_data[["Date", "Close", "CloseD+1", "PercentageD+1", "TrendD+1"]], on='Date', how='left')

In [ ]:
data

In [ ]:
data.describe()

In [ ]:
data[data["PercentageD+1"] < -0.5]

In [ ]:
#data.to_csv("/content/drive/MyDrive/SP500-Predictor/jvdm_data_sources_trends.csv", index=False, float_format="%0.4f")
data.to_csv("../data/jvdm_data_sources_trends.csv", index=False, float_format="%0.4f")

### Examine correlation between TrendD+1 and sentiment as predicted by existing transformer model

In [ ]:
!pip install transformers datasets

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
import pandas as pd

In [ ]:
#data = pd.read_csv("/content/drive/MyDrive/SP500-Predictor/jvdm_data_sources_trends.csv")
data = pd.read_csv("../data/jvdm_data_sources_trends.csv")
data

In [ ]:
from datasets import Dataset
dataset = Dataset.from_pandas(data)

In [ ]:
from transformers import pipeline

model_name = 'mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis'
sentiment_analyzer = pipeline('sentiment-analysis', model=model_name, device=0, batch_size=32)


In [ ]:
def process_batch(batch):
    results = sentiment_analyzer(batch["Headlines"])
    label_map = {"negative": 0, "neutral": 1, "positive": 2}

    # Extract labels and probabilities
    labels = [label_map[result["label"]] for result in results]
    probabilities = [result["score"] for result in results]

    return {"SentimentLabel": labels, "SentimentProb": probabilities}

In [ ]:
dataset = dataset.map(process_batch, batched=True, batch_size=32)

In [ ]:
processed_data = dataset.to_pandas()

In [ ]:
#processed_data.to_csv("/content/drive/MyDrive/SP500-Predictor/jvdm_data_sources_trends_sentiments.csv", index=False, float_format="%0.4f")
processed_data.to_csv("../data/jvdm_data_sources_trends_sentiments.csv", index=False, float_format="%0.4f")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#data = pd.read_csv("/content/drive/MyDrive/SP500-Predictor/jvdm_data_sources_trends_sentiments.csv")
data = pd.read_csv("../data/jvdm_data_sources_trends_sentiments.csv")
data

In [ ]:
data["Correlation"] = data["SentimentLabel"] - data["TrendD+1"]

In [ ]:
print(len(data[(data["Source"] == "CNBC") & (data["Correlation"] == -2)]) / len(data[data["Source"] == "CNBC"]))
print(len(data[(data["Source"] == "CNBC") & (data["Correlation"] == -1)]) / len(data[data["Source"] == "CNBC"]))
print(len(data[(data["Source"] == "CNBC") & (data["Correlation"] == 0)]) / len(data[data["Source"] == "CNBC"]))
print(len(data[(data["Source"] == "CNBC") & (data["Correlation"] == 1)]) / len(data[data["Source"] == "CNBC"]))
print(len(data[(data["Source"] == "CNBC") & (data["Correlation"] == 2)]) / len(data[data["Source"] == "CNBC"]))

In [ ]:
print(len(data[(data["Source"] == "Guardian") & (data["Correlation"] == -2)]) / len(data[data["Source"] == "Guardian"]))
print(len(data[(data["Source"] == "Guardian") & (data["Correlation"] == -1)]) / len(data[data["Source"] == "Guardian"]))
print(len(data[(data["Source"] == "Guardian") & (data["Correlation"] == 0)]) / len(data[data["Source"] == "Guardian"]))
print(len(data[(data["Source"] == "Guardian") & (data["Correlation"] == 1)]) / len(data[data["Source"] == "Guardian"]))
print(len(data[(data["Source"] == "Guardian") & (data["Correlation"] == 2)]) / len(data[data["Source"] == "Guardian"]))

In [ ]:
print(len(data[(data["Source"] == "Reuters") & (data["Correlation"] == -2)]) / len(data[data["Source"] == "Reuters"]))
print(len(data[(data["Source"] == "Reuters") & (data["Correlation"] == -1)]) / len(data[data["Source"] == "Reuters"]))
print(len(data[(data["Source"] == "Reuters") & (data["Correlation"] == 0)]) / len(data[data["Source"] == "Reuters"]))
print(len(data[(data["Source"] == "Reuters") & (data["Correlation"] == 1)]) / len(data[data["Source"] == "Reuters"]))
print(len(data[(data["Source"] == "Reuters") & (data["Correlation"] == 2)]) / len(data[data["Source"] == "Reuters"]))

In [ ]:
data[data["Correlation"] == -2].sample(10, random_state=42)[["Headlines", "TrendD+1", "SentimentLabel"]]

In [ ]:
data[data["Correlation"] == -2].sample(10, random_state=42)[["Headlines", "TrendD+1", "SentimentLabel"]].Headlines.values

In [ ]:
data[data["Correlation"] == 2].sample(10, random_state=42)[["Headlines", "TrendD+1", "SentimentLabel"]].Headlines.values

In [ ]:
data[data["Correlation"] == 0].sample(50, random_state=42)[["Headlines", "TrendD+1", "SentimentLabel"]]

In [ ]:
sample_set = data[data["Correlation"] == 0]

In [ ]:
sample_set

In [ ]:
data["SentimentProb"].describe()

In [ ]:
data[data["SentimentProb"] < 0.9]

In [ ]:
plt.figure(figsize=(10, 6))

# Calculate the normalized count (proportion) for each value within each source
df_normalized = data.groupby('Source')['Correlation'].value_counts(normalize=True).rename('Proportion').reset_index()

# Plot the normalized counts
sns.barplot(data=df_normalized, x='Correlation', y='Proportion', hue='Source')

plt.title('Distribution of TrendD+1/Sentiment correlation per source')
plt.xlabel('Value')
plt.ylabel('Density')
plt.show()

### Conclusion

- There is no strong correlation between headline sentiment and TrendD+1.Finetuning applied to predict headline sentiment is more likely to cause confusion rather than provide a head-start, so it is probably better to use parent model (distilbert/distilroberta-base) in model training
  (added: confirmed by model training tests: use distilroberta-base)
- Using sentiment scores to pre-determine relevance of individual headlines unlikely to be successful due to weak correlation (added: confirmed by model training tests: need other ways to gather more relevant headlines e.g. gather better data; use as-is for now)
- Because increase/decrease are less likely than maintain, the classes are imbalanced (added: balancing classes did not improve accuracy in model training tests: use full dataset for now )
- Despite lots of UK-centric headlines in Guardian dataset, correlation was not significantly different for these headlines vs. the CNBC and Reuters ones: use full dataset for now